In [58]:
from src.explainer.explainer import ArgumentativeExplainer
from src.explainer.framework import ArgumentationFramework

from src.explainer.adjective import BooleanAdjective, PointerAdjective, ComparisonAdjective, MaxRankAdjective, MinRankAdjective
from src.explainer.explanation import Possession, Assumption, PossessionCondition, ConditionalExplanation, CompositeExplanation

## Example search tree:

In [59]:
# Example usage:
class MinMaxNode:
    def __init__(self, id, *, score=None, maximizing_player_turn=True, children=None, score_child=None):
        
        self.id = id

        if score:
            self.score = score
        elif score_child:
            self.score_child = score_child
            self.score = score_child.score
        else:
            raise ValueError("Provide score or score_child.")

        self.children = children or []
        self.parent = None
        self.maximizing_player_turn = maximizing_player_turn
        
        self.is_leaf = True
        if len(self.children) > 0:
            self.is_leaf = False
            for child in children:
                child.parent=self
                child.maximizing_player_turn = not self.maximizing_player_turn
    
    def __str__(self):
        return self.id

# Create a simple game tree
leaf11 = MinMaxNode('leaf11', score=3)
leaf12 = MinMaxNode('leaf12', score=4)
leaf21 = MinMaxNode('leaf21', score=8)
leaf22 = MinMaxNode('leaf22', score=2)
leaf31 = MinMaxNode('leaf21', score=1)
leaf32 = MinMaxNode('leaf22', score=1)

child1 = MinMaxNode('child1', children=[leaf11, leaf12], score_child=leaf11)
child2 = MinMaxNode('child2', children=[leaf21, leaf22], score_child=leaf22)
child3 = MinMaxNode('child3', children=[leaf31, leaf32], score_child=leaf31)

root = MinMaxNode('root', maximizing_player_turn=True, children=[child1, child2, child3], score_child=child1)

## Framework definition

In [60]:
BooleanAdjective("leaf",
        definition = "node.is_leaf")

BooleanAdjective("opponent player turn",
        definition = "not node.maximizing_player_turn")

PointerAdjective("score",
        definition = "node.score",

        explanation = ConditionalExplanation(
            condition = PossessionCondition("leaf"),

            true_explanation = Assumption("Leaf nodes have scores from the evaluation function"),

            false_explanation = CompositeExplanation(
                Assumption("Internal nodes have scores from children"),
                Possession("backtracing child"))
        ))

PointerAdjective("backtracing child",
        definition = "node.score_child",

        explanation = ConditionalExplanation(
            condition = PossessionCondition("opponent player turn"),

            true_explanation = CompositeExplanation(
                Assumption("We assume the opponent will do their best move."),
                Possession("backtracing child", "worst")),

            false_explanation = CompositeExplanation(
                Assumption("On our turn we take the maximum rated move."),
                Possession("backtracing child", "best"))
        ))

ComparisonAdjective("better", "score", ">")
PointerAdjective("siblings", definition = "node.parent.children")

MaxRankAdjective("best", "better", "siblings")
MinRankAdjective("worst", "better", "siblings") # worst= not better than any sibling

In [61]:
knowledgebase = ArgumentationFramework()

knowledgebase.add_adjective( 
    BooleanAdjective("leaf",
        definition = "node.is_leaf")
)

knowledgebase.add_adjective( 
    PointerAdjective("score",
        definition = "node.score",

        explanation = ConditionalExplanation(
            condition = PossessionCondition("leaf"),
            true_explanation = Assumption("Leaf nodes have scores from the evaluation function"),
            false_explanation = CompositeExplanation(
                Assumption("Internal nodes have scores from children"),
                Possession("backtracing child"))
        ))
)

knowledgebase.add_adjective(
    BooleanAdjective("opponent player turn",
        definition = "not node.maximizing_player_turn")
)

knowledgebase.add_adjective( 
    PointerAdjective("backtracing child",
        definition = "node.score_child",
        # alternative definition for backtracing child:
        # min(node.children, key=lambda child: child.score),

        explanation = ConditionalExplanation(
            condition = PossessionCondition("opponent player turn"),
            true_explanation = CompositeExplanation(
                Assumption("We assume the opponent will do their best move."),
                Possession("backtracing child", "worst")),
            false_explanation = CompositeExplanation(
                Assumption("On our turn we take the maximum rated move."),
                Possession("backtracing child", "best"))
        ))
)

knowledgebase.add_adjective(
    ComparisonAdjective("better", "score", ">")
)

knowledgebase.add_adjective( 
    PointerAdjective("siblings",
        definition = "[sibling for sibling in node.parent.children if sibling is not node]")
)

knowledgebase.add_adjective(
    MaxRankAdjective("best", "better", "siblings")
)
knowledgebase.add_adjective(
    MinRankAdjective("worst", "better", "siblings")
)

In [62]:
print(knowledgebase)

Propositions:
node is leaf
node has score = ?
node is opponent player turn
node has backtracing child = ?
node1 is better than node2
node has siblings = ?
node is best
node is worst


Implications:
[(assumption) Definition of "leaf" is "node.is_leaf" → node is leaf]

[[(assumption) Definition of "leaf" is "node.is_leaf" → node is leaf]
 ∧ (assumption) Leaf nodes have scores from the evaluation function
 ∨ [(assumption) Definition of "leaf" is "node.is_leaf" → ¬(node is leaf)]
 ∧ (assumption) Internal nodes have scores from children ∧ [[(assumption) Definition of "opponent player turn" is "not node.maximizing_player_turn" → node is opponent player turn]
 ∧ (assumption) We assume the opponent will do their best move. ∧ [(assumption) By definition a node is "worst" if it's ¬("better") compared to all nodes among "siblings" ∧ Node ¬(better) than all nodes in siblings → node is worst]

 ∨ [(assumption) Definition of "opponent player turn" is "not node.maximizing_player_turn" → ¬(node is opp

In [63]:
explainer = ArgumentativeExplainer(knowledgebase)

explainer.set_tree_search_motivation(lambda root: root.children, "best")

## Set your settings

In [64]:
settings = {
            'explanation_depth': 3 ,
            'assumptions_verbosity': 'verbose',
            'repeat_explanations': False
        }

knowledgebase.configure_settings(settings)

## You can now explain nodes' properties

In [65]:
print(explainer.explain_adjective(child1, "best"))

[(assumption) By definition a node is "best" if it's "better" compared to all nodes among "siblings" ∧ [[[(assumption) Definition of "leaf" is "node.is_leaf" → ¬(child1 is leaf)]
 ∧ (assumption) Internal nodes have scores from children ∧ [[(assumption) Definition of "opponent player turn" is "not node.maximizing_player_turn" → child1 is opponent player turn]
 ∧ (assumption) We assume the opponent will do their best move. ∧ [(assumption) By definition a node is "worst" if it's ¬("better") compared to all nodes among "siblings" ∧ [[leaf11 is leaf ∧ (assumption) Leaf nodes have scores from the evaluation function → leaf11 has score = 3]
 ∧ [leaf12 is leaf ∧ (assumption) Leaf nodes have scores from the evaluation function → leaf12 has score = 4]
 → [(assumption) By definition, node1 is "better" than node2 if node1 score > node2 score → ¬(leaf11 is better than leaf12)]
]
 → leaf11 is worst]
 → child1 has backtracing child = leaf11]
 → child1 has score = 3]
 ∧ [¬(child2 is leaf) ∧ (assumptio

In [66]:
knowledgebase.settings.repeat_explanations = True

In [67]:
print(explainer.explain_adjective(child1, "best"))

[(assumption) By definition a node is "best" if it's "better" compared to all nodes among "siblings" ∧ [[[(assumption) Definition of "leaf" is "node.is_leaf" → ¬(child1 is leaf)]
 ∧ (assumption) Internal nodes have scores from children ∧ [[(assumption) Definition of "opponent player turn" is "not node.maximizing_player_turn" → child1 is opponent player turn]
 ∧ (assumption) We assume the opponent will do their best move. ∧ [(assumption) By definition a node is "worst" if it's ¬("better") compared to all nodes among "siblings" ∧ [[[(assumption) Definition of "leaf" is "node.is_leaf" → leaf11 is leaf]
 ∧ (assumption) Leaf nodes have scores from the evaluation function → leaf11 has score = 3]
 ∧ [[(assumption) Definition of "leaf" is "node.is_leaf" → leaf12 is leaf]
 ∧ (assumption) Leaf nodes have scores from the evaluation function → leaf12 has score = 4]
 → [(assumption) By definition, node1 is "better" than node2 if node1 score > node2 score → ¬(leaf11 is better than leaf12)]
]
 → leaf

In [68]:
knowledgebase.settings.explanation_depth = 1

In [69]:
print(explainer.explain_adjective(child1, "best"))

child1 is best


In [70]:
knowledgebase.settings.explanation_depth = 3
knowledgebase.settings.repeat_explanations = False

In [71]:
print(explainer.explain_adjective(child1, "worst"))

[(assumption) By definition a node is "worst" if it's ¬("better") compared to all nodes among "siblings" ∧ [[[(assumption) Definition of "leaf" is "node.is_leaf" → ¬(child1 is leaf)]
 ∧ (assumption) Internal nodes have scores from children ∧ [[(assumption) Definition of "opponent player turn" is "not node.maximizing_player_turn" → child1 is opponent player turn]
 ∧ (assumption) We assume the opponent will do their best move. ∧ leaf11 is worst → child1 has backtracing child = leaf11]
 → child1 has score = 3]
 ∧ [¬(child2 is leaf) ∧ (assumption) Internal nodes have scores from children ∧ [child2 is opponent player turn ∧ (assumption) We assume the opponent will do their best move. ∧ leaf22 is worst → child2 has backtracing child = leaf22]
 → child2 has score = 2]
 → [(assumption) By definition, node1 is "better" than node2 if node1 score > node2 score → child1 is better than child2]
]
 ∧ [[¬(child1 is leaf) ∧ (assumption) Internal nodes have scores from children ∧ [child1 is opponent pla

In [72]:
print(explainer.explain_adjective(leaf11, "worst"))

[(assumption) By definition a node is "worst" if it's ¬("better") compared to all nodes among "siblings" ∧ [[[(assumption) Definition of "leaf" is "node.is_leaf" → leaf11 is leaf]
 ∧ (assumption) Leaf nodes have scores from the evaluation function → leaf11 has score = 3]
 ∧ [leaf12 is leaf ∧ (assumption) Leaf nodes have scores from the evaluation function → leaf12 has score = 4]
 → [(assumption) By definition, node1 is "better" than node2 if node1 score > node2 score → ¬(leaf11 is better than leaf12)]
]
 → leaf11 is worst]



In [73]:
print(explainer.explain_adjective(child2, "best"))

[(assumption) By definition a node is "best" if it's "better" compared to all nodes among "siblings" ∧ [[[(assumption) Definition of "leaf" is "node.is_leaf" → ¬(child2 is leaf)]
 ∧ (assumption) Internal nodes have scores from children ∧ [[(assumption) Definition of "opponent player turn" is "not node.maximizing_player_turn" → child2 is opponent player turn]
 ∧ (assumption) We assume the opponent will do their best move. ∧ [(assumption) By definition a node is "worst" if it's ¬("better") compared to all nodes among "siblings" ∧ [[leaf22 is leaf ∧ (assumption) Leaf nodes have scores from the evaluation function → leaf22 has score = 2]
 ∧ [leaf21 is leaf ∧ (assumption) Leaf nodes have scores from the evaluation function → leaf21 has score = 8]
 → [(assumption) By definition, node1 is "better" than node2 if node1 score > node2 score → ¬(leaf22 is better than leaf21)]
]
 → leaf22 is worst]
 → child2 has backtracing child = leaf22]
 → child2 has score = 2]
 ∧ [¬(child1 is leaf) ∧ (assumptio

In [74]:
print(explainer.explain_adjective(root, "backtracing child"))

[[(assumption) Definition of "opponent player turn" is "not node.maximizing_player_turn" → ¬(root is opponent player turn)]
 ∧ (assumption) On our turn we take the maximum rated move. ∧ [(assumption) By definition a node is "best" if it's "better" compared to all nodes among "siblings" ∧ [[[(assumption) Definition of "leaf" is "node.is_leaf" → ¬(child1 is leaf)]
 ∧ (assumption) Internal nodes have scores from children ∧ [child1 is opponent player turn ∧ (assumption) We assume the opponent will do their best move. ∧ [(assumption) By definition a node is "worst" if it's ¬("better") compared to all nodes among "siblings" ∧ [[leaf11 is leaf ∧ (assumption) Leaf nodes have scores from the evaluation function → leaf11 has score = 3]
 ∧ [leaf12 is leaf ∧ (assumption) Leaf nodes have scores from the evaluation function → leaf12 has score = 4]
 → [(assumption) By definition, node1 is "better" than node2 if node1 score > node2 score → ¬(leaf11 is better than leaf12)]
]
 → leaf11 is worst]
 → chil

In [75]:
print(explainer.explain_adjective(root, "score"))

[[(assumption) Definition of "leaf" is "node.is_leaf" → ¬(root is leaf)]
 ∧ (assumption) Internal nodes have scores from children ∧ [[(assumption) Definition of "opponent player turn" is "not node.maximizing_player_turn" → ¬(root is opponent player turn)]
 ∧ (assumption) On our turn we take the maximum rated move. ∧ [(assumption) By definition a node is "best" if it's "better" compared to all nodes among "siblings" ∧ [[¬(child1 is leaf) ∧ (assumption) Internal nodes have scores from children ∧ [child1 is opponent player turn ∧ (assumption) We assume the opponent will do their best move. ∧ [(assumption) By definition a node is "worst" if it's ¬("better") compared to all nodes among "siblings" ∧ [[leaf11 is leaf ∧ (assumption) Leaf nodes have scores from the evaluation function → leaf11 has score = 3]
 ∧ [leaf12 is leaf ∧ (assumption) Leaf nodes have scores from the evaluation function → leaf12 has score = 4]
 → [(assumption) By definition, node1 is "better" than node2 if node1 score > n

Track down that Not before the "Considering your definition of leaf"

In [76]:
print(explainer.query_explanation(root, "Why is child 1 maxoptimal?"))

None
